In [27]:
import numpy as np
import pandas as pd

In [28]:
__author__ = 'Chris Potts'


class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        The temperature parameter. Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        return rownorm(self.lexicon * self.prior)

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        spk = self.speaker().T
        return rownorm(spk * self.prior)


def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Examples"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses']
    states = ['r1', 'r2']
    lex = pd.DataFrame([
        [0.0, 1.0],
        [1.0, 1.0]], index=msgs, columns=states)

### Задание 3: референциальная игра

#### Задание 3.1:

Приведите скалярную импликатуру, которую можно вывести из таблицы прагматического слушающего.

In [29]:
# Лексикон задания 3

msgs1 = ['шляпа', 'очки', 'усы']
states1 = ['r1', 'r2', 'r3']
lex1 = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 1.0, 0.0]], index=msgs1, columns=states1)

In [30]:
# Таблицы задания 3.1

print("="*70 + "\nРеференциальная игра 1\n")
basic_mod = RSA(lexicon=lex1, prior=[1/3, 1/3, 1/3], costs=[0.0, 0.0, 0.0])

display_reference_game(basic_mod)

print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

Референциальная игра 1



,r1,r2,r3,costs
шляпа,1.000000,1.0,0.0,0.0
очки,0.000000,0.0,1.0,0.0
усы,0.000000,1.0,0.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Literal listener


,r1,r2,r3
шляпа,0.5,0.5,0.0
очки,0.0,0.0,1.0
усы,0.0,1.0,0.0



Pragmatic speaker


,шляпа,очки,усы
r1,1.000000,0.0,0.000000
r2,0.333333,0.0,0.666667
r3,0.000000,1.0,0.000000



Pragmatic listener


,r1,r2,r3
шляпа,0.75,0.25,0.0
очки,0.00,0.00,1.0
усы,0.00,1.00,0.0


Вывод следующий: прагматичный слушатель с большей вероятностью (0.75) соотнесёт высказывание «шляпа» с объектом r1, нежели с объектом r2 (0.25). Таким образом, высказывание «шляпа» с большей вероятность интерпретируется как «шляпа, но не усы», что и является импликатурой.

#### Задание 3.2
Как поменяется импликатура, если поменять исходные вероятности на P(r1) = 0.1, P(r2) = P(r3) = 0.45?

In [31]:
# Задание 3.2

print("="*70 + "\nРеференциальная игра 1\n")
basic_mod = RSA(lexicon=lex1, prior=[0.1, 0.45, 0.45], costs=[0.0, 0.0, 0.0])

display_reference_game(basic_mod)

print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

Референциальная игра 1



,r1,r2,r3,costs
шляпа,1.0,1.0,0.0,0.0
очки,0.0,0.0,1.0,0.0
усы,0.0,1.0,0.0,0.0
prior,0.1,0.45,0.45,
alpha,1.0,,,



Literal listener


,r1,r2,r3
шляпа,0.181818,0.818182,0.0
очки,0.000000,0.000000,1.0
усы,0.000000,1.000000,0.0



Pragmatic speaker


,шляпа,очки,усы
r1,1.00,0.0,0.00
r2,0.45,0.0,0.55
r3,0.00,1.0,0.00



Pragmatic listener


,r1,r2,r3
шляпа,0.330579,0.669421,0.0
очки,0.000000,0.000000,1.0
усы,0.000000,1.000000,0.0


Как видно из таблицы прагматического слушателя, при изменении исходных вероятностей, импликатура меняется следующим образом: услышав высказывание «шляпа», прагматический слушатель с большей вероятностью (~ 0.66) выберет объект r2, нежели объект r1 (вероятность ~0.33).

Предположу, что возникает вероятность коммуникацонной неудачи: при высказывании "шляпа" слушатель имеет больше шансов неверно определить референцию и выбрать объект r2. Таким образом, импликатура не считается в большинстве случаев.

Высказывание «усы» по-прежнему будет ассоциироваться с объектом r2.

### Задание 4: референциальная игра с 4 объектами

Насколько я понял, задание не предполагает создание концептуально новой версии игры, поскольку RSA модель работала как для примеров с фигурами и цветами, так и для примеров с человечками и аксессуарами.

Основываясь на этом предположении, я добавил ещё один аксессуар "галстук", ещё одно состояние/объект r4, ещё одно высказывание "галстук", и расширил матрицу. Распределение состояний произвольное.

In [32]:
# Лексикон задания 4

msgs2 = ['шляпа', 'очки', 'усы', 'галстук']
states2 = ['r1', 'r2', 'r3', 'r4']
lex2 = pd.DataFrame([
    [1.0, 1.0, 0.0, 0.0],
    [0.0, 0.0, 1.0, 1.0],
    [0.0, 1.0, 0.0, 0.0],
    [0.0, 0.0, 1.0, 0.0]], index=msgs2, columns=states2)

In [33]:
print("="*70 + "\nРеференциальная игра с 4 объектами\n")
basic_mod = RSA(lexicon=lex2, prior=[0.25, 0.25, 0.25, 0.25], 
                costs=[0.0, 0.0, 0.0, 0.0])

display_reference_game(basic_mod)

print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

Референциальная игра с 4 объектами



,r1,r2,r3,r4,costs
шляпа,1.00,1.0,0.0,0.0,0.0
очки,0.00,0.0,1.0,1.0,0.0
усы,0.00,1.0,0.0,0.0,0.0
галстук,0.00,0.0,1.0,0.0,0.0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Literal listener


,r1,r2,r3,r4
шляпа,0.5,0.5,0.0,0.0
очки,0.0,0.0,0.5,0.5
усы,0.0,1.0,0.0,0.0
галстук,0.0,0.0,1.0,0.0



Pragmatic speaker


,шляпа,очки,усы,галстук
r1,1.000000,0.000000,0.000000,0.000000
r2,0.333333,0.000000,0.666667,0.000000
r3,0.000000,0.333333,0.000000,0.666667
r4,0.000000,1.000000,0.000000,0.000000



Pragmatic listener


,r1,r2,r3,r4
шляпа,0.75,0.25,0.00,0.00
очки,0.00,0.00,0.25,0.75
усы,0.00,1.00,0.00,0.00
галстук,0.00,0.00,1.00,0.00


Наблюдаем две импликатуры:

* высказывание "шляпа" интерпретируется слушателем как "шляпа, но не усы" и с большей вероятностью связывается с объектом r1 (0.75), чем с r2 (0.25);
* высказывание "очки" интерпретируется слушателем как "очки, но не галстук" и с большей вероятностью связывается с объектом r4 (0.75), чем с r3 (0.25);

#### Задание 4.2: изменение параметров цены высказывания и параметра альфа

#### Изменение цены
Изменим цену высказывания "шляпа" на -13. Значение подобрано эмпирически, с целью снять импликатуру.

In [34]:
print("="*70 + "\nЦена высказывания \"шляпа\"\n")
basic_mod = RSA(lexicon=lex2, prior=[0.25, 0.25, 0.25, 0.25], 
                costs=[-13.0, 0.0, 0.0, 0.0])

display_reference_game(basic_mod)

print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

Цена высказывания "шляпа"



,r1,r2,r3,r4,costs
шляпа,1.00,1.0,0.0,0.0,-13.0
очки,0.00,0.0,1.0,1.0,0.0
усы,0.00,1.0,0.0,0.0,0.0
галстук,0.00,0.0,1.0,0.0,0.0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Literal listener


,r1,r2,r3,r4
шляпа,0.5,0.5,0.0,0.0
очки,0.0,0.0,0.5,0.5
усы,0.0,1.0,0.0,0.0
галстук,0.0,0.0,1.0,0.0



Pragmatic speaker


,шляпа,очки,усы,галстук
r1,1.000000,0.000000,0.000000,0.000000
r2,0.000001,0.000000,0.999999,0.000000
r3,0.000000,0.333333,0.000000,0.666667
r4,0.000000,1.000000,0.000000,0.000000



Pragmatic listener


,r1,r2,r3,r4
шляпа,0.999999,0.000001,0.00,0.00
очки,0.000000,0.000000,0.25,0.75
усы,0.000000,1.000000,0.00,0.00
галстук,0.000000,0.000000,1.00,0.00


С уменьшением цены высказывания "шляпа" вероятность верного считывания импликатуры стремится к 100%, но не достигает этого значения, позволяя предположить, что остаётся незначительный шанс неверной интерпретации.

**Вопрос**: можем ли мы пренебречь таким шансом (0.000001) в рамках данной модели?



Схожие результаты наблюдаем при изменении цены двух высказываний:

In [35]:
print("="*70 + "\nЦена высказывания \"шляпа\" и \"очки\"\n")
basic_mod = RSA(lexicon=lex2, prior=[0.25, 0.25, 0.25, 0.25], 
                costs=[-13.0, -13.0, 0.0, 0.0])

display_reference_game(basic_mod)

print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

Цена высказывания "шляпа" и "очки"



,r1,r2,r3,r4,costs
шляпа,1.00,1.0,0.0,0.0,-13.0
очки,0.00,0.0,1.0,1.0,-13.0
усы,0.00,1.0,0.0,0.0,0.0
галстук,0.00,0.0,1.0,0.0,0.0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Literal listener


,r1,r2,r3,r4
шляпа,0.5,0.5,0.0,0.0
очки,0.0,0.0,0.5,0.5
усы,0.0,1.0,0.0,0.0
галстук,0.0,0.0,1.0,0.0



Pragmatic speaker


,шляпа,очки,усы,галстук
r1,1.000000,0.000000,0.000000,0.000000
r2,0.000001,0.000000,0.999999,0.000000
r3,0.000000,0.000001,0.000000,0.999999
r4,0.000000,1.000000,0.000000,0.000000



Pragmatic listener


,r1,r2,r3,r4
шляпа,0.999999,0.000001,0.000000,0.000000
очки,0.000000,0.000000,0.000001,0.999999
усы,0.000000,1.000000,0.000000,0.000000
галстук,0.000000,0.000000,1.000000,0.000000


#### Изменение параметра alpha

In [36]:
print("="*70 + "\nИзменение alpha\n")
alpha_mod = RSA(lexicon=lex2, prior=[0.25, 0.25, 0.25, 0.25], 
                costs=[0.0, 0.0, 0.0, 0.0], alpha=17.0)

display_reference_game(alpha_mod)

print("\nLiteral listener")
display(alpha_mod.literal_listener())

print("\nPragmatic speaker")
display(alpha_mod.speaker())

print("\nPragmatic listener")
display(alpha_mod.listener())

Изменение alpha



,r1,r2,r3,r4,costs
шляпа,1.00,1.0,0.0,0.0,0.0
очки,0.00,0.0,1.0,1.0,0.0
усы,0.00,1.0,0.0,0.0,0.0
галстук,0.00,0.0,1.0,0.0,0.0
prior,0.25,0.25,0.25,0.25,
alpha,17.00,,,,



Literal listener


,r1,r2,r3,r4
шляпа,0.5,0.5,0.0,0.0
очки,0.0,0.0,0.5,0.5
усы,0.0,1.0,0.0,0.0
галстук,0.0,0.0,1.0,0.0



Pragmatic speaker


,шляпа,очки,усы,галстук
r1,1.000000,0.000000,0.000000,0.000000
r2,0.000008,0.000000,0.999992,0.000000
r3,0.000000,0.000008,0.000000,0.999992
r4,0.000000,1.000000,0.000000,0.000000



Pragmatic listener


,r1,r2,r3,r4
шляпа,0.999992,0.000008,0.000000,0.000000
очки,0.000000,0.000000,0.000008,0.999992
усы,0.000000,1.000000,0.000000,0.000000
галстук,0.000000,0.000000,1.000000,0.000000


С ростом параметра alpha, который отражает оптимальность и рациональность говорящего, наблюдается схожий результат - импликатура *практически* исчезает.